In [ ]:
import locale
import traceback
import pymysql
import dash
from dash import html
from dash.dependencies import Input, Output, State
from rol_usuario_funciones import *
from rol_administrador_funciones import *
from layout import *
from login import *

# Setea la variable LC_ALL al conjunto de código UTF-8 con descripción español España
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

colors = {"graphBackground": "#F5F5F5", "background": "#ffffff", "text": "#000000"}

not_uploaded_files = []
uploaded_files = []

Relizamos import de las librerías necesarias, creamos la app y procedemos con la llamada a la base de datos y la creación de las listas para los desplegables.

In [ ]:
#Creamos dataframe
df = crear_dataframe()

Una vez creados todos los desplegables, creamos el layout.

In [ ]:
app.layout =  html.Div(
            children=[
               generar_layout_usuario(df),
               generar_layout_login(),
               generar_layout_administrador(),
               generar_layout_elegir_rol()
            ],
        )       

Login

In [ ]:
@app.callback(
    Output("layout-login", "style"),
    Output("layout-usuario", "style"),
    Output("layout-administrador", "style"),
    Output("layout-elegir-rol", "style"),
    Output("login-error-message", "children"),
    Input("login-button", "n_clicks"),
    Input("elegir-usuario-button", "n_clicks"),
    Input("elegir-administrador-button", "n_clicks"),
    [
        State("username-input", "value"),
        State("password-input", "value")
    ],
)

def login(n_clicks, n_clicks_rol_usuario, n_clicks_rol_administrador, username, password):    
    if n_clicks and not n_clicks_rol_usuario and not n_clicks_rol_administrador:
        
        if comprobar_constraseña(username, password) == 1:
            #Elegir rol visible
            return {"display": "none"}, {"display": "none"}, {"display": "none"}, {"display": "block"},""
        elif comprobar_constraseña(username, password) == 0:
            #Rol usuario visible
            return {"display": "none"}, {"display": "block"}, {"display": "none"}, {"display": "none"},""
        else:
            #Mostramos error en login
            return {"display": "block"}, {"display": "none"}, {"display": "none"}, {"display": "none"},"Los datos introducidos no son válidos."
        
    elif n_clicks_rol_usuario:
        #Rol usuario visible
        return {"display": "none"}, {"display": "block"}, {"display": "none"}, {"display": "none"},""
    
    elif n_clicks_rol_administrador:
        #Rol administrador visible
        return {"display": "none"}, {"display": "none"}, {"display": "block"}, {"display": "none"},""
    
    else:
        #Nos quedamos en login
        return {"display": "block"}, {"display": "none"}, {"display": "none"}, {"display": "none"},""


Creamos las funciones callback necesarias para rol usuario.

In [ ]:
@app.callback(
    [
        Output("map", "children"),
        Output("output-data-1", "children"),
        Output("output-data-2", "children"),
        Output("output-data-3", "children"),
        Output("output-data-4", "children"),
        Output("output-data-message", "children"),
        Output("output-data-message-mapa", "children")
    ],
    [
        Input("dropdownTramos", "value"),
        Input("dropdownMeses", "value"),
        Input("dropdownAnios", "value"),
        Input("dropdownTipos", "value"),
        Input("dropdownModus", "value"),
        Input("dropdownCalificacion", "value"),
        Input("dropdownDistrito", "value"),
    ],
)

def upload_and_write_data(dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito):
    return generar_figuras(df, dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito)


@app.callback(
    Output("informe-output-message", "children"),
    Input("generar-informe-button", "n_clicks"),
    [
        State("dropdownTramos", "value"),
        State("dropdownMeses", "value"),
        State("dropdownAnios", "value"),
        State("dropdownTipos", "value"),
        State("dropdownModus", "value"),
        State("dropdownCalificacion", "value"),
        State("dropdownDistrito", "value"),
    ],
)

def generar_informe(n_clicks, dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito):
    if n_clicks:
        return generar_informe_pdf(df, n_clicks, dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito)


Creamos las funciones callback necesarias para rol administrador.

In [ ]:
@app.callback(
    Output("output-data-not-upload", "children"),
    Input("upload-data", "filename"),
    Input("upload-data", "contents")
)

def upload_file(filename, contents):
    if filename is not None:
        divs = [
            html.Div([
                f"El archivo {file} ha sido subido.",
                #html.Button('Delete file', id=f'cancel-{file}', style={'display': 'inline-block', 'marginLeft': '10px'}, n_clicks=0),
            ]) 
        for file in filename]


        return divs
    else:
        return html.Div(["Ningún archivo ha sido subido."])
    
@app.callback(
    Output("output-data-upload", "children"),
    Input("submit-button", "n_clicks"), 
    State("upload-data", "filename"),
    State("upload-data", "contents"),
    #*[State(f'cancel-{file}', 'n_clicks') for file in not_uploaded_files]
)

def upload_and_write_data(n_clicks, filenames, contents):
    try:
        if(n_clicks > 0):
            listdenuncias = []
            if contents is not None:
                listdenuncias = insert_data(contents)

                # Return a message indicating that the data was written to the database
                for index, file in enumerate(filenames):
                    if listdenuncias[index] == 0:
                        not_uploaded_files.append(file)
                    else:
                        uploaded_files.append(file)
                        
                divs = [html.Div([f"El archivo {file} ha sido resgitrado con éxito en la base de datos."]) for file in uploaded_files]
                divs2 = [html.Div([f"El archivo {file} NO CONTIENE LAS CABECERAS ADECUADAS."]) for file in not_uploaded_files]
                return divs + divs2
    except pymysql.err.InterfaceError as e:
        print(e)
        traceback.print_exc()
        return html.Div(["Ha habido un error estableciendo la conexión con la base de datos, disculpe las molestias."])
    except Exception as e:
        print(e)
        traceback.print_exc()
        return html.Div(["Ha habido un error procesando los datos, disculpe las molestias."])

Por último, ejecutamos la aplicación

In [14]:
app.run_server(port="8051")